In [70]:
# Load the pyomo package
!pip install pyomo

In [53]:
# Import package
import pandas as pd
from pyomo.environ import *

# Read the dara
rooms_df = pd.read_excel(r'C:\Users\caowe\Desktop\Timetabling_KB_Rooms.xlsx')  # Contains room IDs and sizes and other details
courses_df = pd.read_excel(r'C:\Users\caowe\Desktop\All_S1_Y1234.xlsx')  # Contains course IDs, expected sizes, and other details

In [54]:
# Create a model
model = ConcreteModel()

# Define parameters based on data
room_sizes = rooms_df.set_index('ROOM NAME')['CAP'].to_dict()
course_sizes = courses_df.set_index('Course')['Average'].to_dict()
max_hours = courses_df.set_index('Course')['Hour_week'].to_dict()

# Define sets for times, days, weeks, rooms, courses and time period
model.times = Set(initialize=['9AM', '10AM', '11AM', '12PM', '1PM', '2PM', '3PM', '4PM', '5PM'])
model.days = Set(initialize=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
model.weeks = Set(initialize=['week1','week2','week3','week4','week5','week6','week7','week8','week9','week10'])
model.rooms = Set(initialize=room_sizes.keys())
model.courses = Set(initialize=course_sizes.keys())
model.time_periods = Set(initialize=[(day, time) for day in model.days for time in model.times])

# Define parameters
model.max_hours_per_week = Param(model.courses, initialize=max_hours)
model.room_size = Param(model.rooms, initialize=room_sizes, within=NonNegativeIntegers)
model.max_hours_per_course = Param(model.courses, initialize=max_hours)
model.expected_course_size = Param(model.courses, initialize=course_sizes, within=NonNegativeIntegers)

# Define binary decision variables
model.course_in_room = Var(model.courses, model.rooms, within=Binary)
model.course_scheduled = Var(model.courses, model.rooms, model.days, model.times, model.weeks, within=Binary)
model.selected_time_slot = Var(model.courses, model.times, within=Binary)
model.course_in_room_at_time = Var(model.courses, model.rooms, model.time_periods, within=Binary) 

In [56]:
# Define weights for the objective function to prioritize minimizing total hours and room size differences
weight_total_hours = 0.6  # Adjust this weight to prioritize minimizing total hours
weight_room_size_difference = 0.4  # Adjust this weight to prioritize minimizing room size difference

# Define the objective function
def combined_objective_rule(model):
    # Calculate the total hours 
    total_hours = sum(model.course_scheduled[c, r, d, t, w] for c in model.courses for r in model.rooms for d in model.days for t in model.times for w in model.weeks)
    # Calculate the room size differences
    room_size_difference = sum(abs(model.room_size[r] - model.expected_course_size[c]) * model.course_scheduled[c, r, d, t, w] for c in model.courses for r in model.rooms for d in model.days for t in model.times for w in model.weeks)
    
    return weight_total_hours * total_hours + weight_room_size_difference * room_size_difference # Combine the total hours and room size differences

# Set the objective function
model.objective = Objective(rule=combined_objective_rule, sense=minimize)

In [58]:
# Constraint to ensure no room is double-booked at any time
def room_not_double_booked_rule(model, r, day, time):
    return sum(model.course_in_room_at_time[c, r, (day, time)] for c in model.courses) <= 1

model.room_not_double_booked_constraint = Constraint(model.rooms, model.days, model.times, rule=room_not_double_booked_rule)

In [62]:
# Constraint to ensure if a time slot is selected for a course, it must be used across all days and weeks
def enforce_selected_time_slot_rule(model, c, t):
    return sum(model.course_scheduled[c, r, d, t, w] for r in model.rooms for d in model.days for w in model.weeks) >= model.selected_time_slot[c, t] * len(model.days) * len(model.weeks)

model.enforce_selected_time_slot_constraint = Constraint(model.courses, model.times, rule=enforce_selected_time_slot_rule)

In [63]:
# Constraint to ensure that each course must have exactly one time slot selected
def single_time_slot_rule(model, c):
    return sum(model.selected_time_slot[c, t] for t in model.times) == 1

model.single_time_slot_constraint = Constraint(model.courses, rule=single_time_slot_rule)

In [69]:
# Constraint to ensure that there is no overlap in scheduling for each room, day, time, and week
def no_overlap_rule(model, r, d, t, w):
    return sum(model.course_scheduled[c, r, d, t, w] for c in model.courses) <= 1

model.no_overlap_constraint = Constraint(model.rooms, model.days, model.times, model.weeks, rule=no_overlap_rule)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [65]:
# Constraint to ensure that for any given room, day, time, and week, at most one course is scheduled
def no_same_room_time_overlap_rule(model, r, d, t, w):
    return sum(model.course_scheduled[c, r, d, t, w] for c in model.courses) <= 1

model.no_same_room_time_overlap_constraint = Constraint(model.rooms, model.days, model.times, model.weeks, rule=no_same_room_time_overlap_rule)

In [66]:
solver = SolverFactory('glpk')  # Using GLPK solver
result = solver.solve(model, tee=True)  # tee=True for verbose output

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\caowe\AppData\Local\Temp\tmp1e3cpsps.glpk.raw --wglp C:\Users\caowe\AppData\Local\Temp\tmpclik7o2c.glpk.glp
 --cpxlp C:\Users\caowe\AppData\Local\Temp\tmphu8ao3bk.pyomo.lp
Reading problem data from 'C:\Users\caowe\AppData\Local\Temp\tmphu8ao3bk.pyomo.lp'...
C:\Users\caowe\AppData\Local\Temp\tmphu8ao3bk.pyomo.lp:7224844: warning: lower bound of variable 'x2' redefined
C:\Users\caowe\AppData\Local\Temp\tmphu8ao3bk.pyomo.lp:7224844: warning: upper bound of variable 'x2' redefined
55330 rows, 1493388 columns, 4208256 non-zeros
1493388 integer variables, all of which are binary
8718232 lines were read
Writing problem data to 'C:\Users\caowe\AppData\Local\Temp\tmpclik7o2c.glpk.glp'...
7169507 lines were written
GLPK Integer Optimizer, v4.65
55330 rows, 1493388 columns, 4208256 non-zeros
1493388 integer variables, all of which are binary
Preprocessing...
55330 rows, 1493388 columns, 4208256 non-zer

In [67]:
# Display the result
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found. Printing schedule...")
    for c in model.courses:
        print(f"Schedule for Course {c}:")
        for w in model.weeks:
            for d in model.days:
                for t in model.times:
                    for r in model.rooms:
                        if model.course_scheduled[c, r, d, t, w].value == 1:  # Assuming the variable is binary
                            print(f" - Week {w}, {d} at {t}, Room {r}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print('No feasible solution found.')
else:
    print('Solver Status:', result.solver.status)

Optimal solution found. Printing schedule...
Schedule for Course Fundamentals of Algebra and Calculus:
 - Week week1, Monday at 5PM, Room JBB_Theatre 250
 - Week week1, Tuesday at 5PM, Room JBB_Theatre 250
 - Week week1, Wednesday at 5PM, Room JBB_Theatre 250
 - Week week1, Thursday at 5PM, Room JBB_Theatre 250
 - Week week1, Friday at 5PM, Room JBB_Theatre 250
 - Week week2, Monday at 5PM, Room JBB_Theatre 250
 - Week week2, Tuesday at 5PM, Room JBB_Theatre 250
 - Week week2, Wednesday at 5PM, Room JBB_Theatre 250
 - Week week2, Thursday at 5PM, Room JBB_Theatre 250
 - Week week2, Friday at 5PM, Room JBB_Theatre 250
 - Week week3, Monday at 5PM, Room JBB_Theatre 250
 - Week week3, Tuesday at 5PM, Room JBB_Theatre 250
 - Week week3, Wednesday at 5PM, Room JBB_Theatre 250
 - Week week3, Thursday at 5PM, Room JBB_Theatre 250
 - Week week3, Friday at 5PM, Room JBB_Theatre 250
 - Week week4, Monday at 5PM, Room JBB_Theatre 250
 - Week week4, Tuesday at 5PM, Room JBB_Theatre 250
 - Week wee